# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import pandas as pd
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment,kgovindmlwk,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "govindk-nvidia-gpunode"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


In [4]:
# create a TabularDataset from a dataset

ds = Dataset.get_by_name(ws, name='uci_heart_disease')
df = ds.to_pandas_dataframe()

x = df.dropna()
y = x.pop('target')

x_train, x_test, y_train, y_test = train_test_split(x, y)
dataset = pd.concat([x_train,y_train],axis=1)
type(dataset)

pandas.core.frame.DataFrame

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
project_folder = './Udacity-Capstone-Project-master'
automl_settings = {
    "compute_target": compute_target,
    "task": 'classification',
    "iterations": 30,
    "experiment_timeout_minutes": 15,
    "primary_metric": 'accuracy',
    "path": project_folder,
    "label_column_name": 'target',
    "n_cross_validations": 5,
    "max_concurrent_iterations": 30,
    "featurization": 'auto',
    "debug_log": "automl_errors.log",
    "enable_early_stopping": True
}


# TODO: Put your automl config here
automl_config = AutoMLConfig(training_data=ds, **automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_8f8525ee-e6a1-462e-a49a-e549b66fe06f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_8f8525ee-e6a1-462e-a49a-e549b66fe06f',
 'target': 'govindk-nvidia-gpunode',
 'status': 'Completed',
 'startTimeUtc': '2021-08-24T03:29:58.635391Z',
 'endTimeUtc': '2021-08-24T03:39:31.757813Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'govindk-nvidia-gpunode',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"417646f6-643f-4fcd-96b8-77997bfdfac3","resource_group":"kgovind-ml-wksp","workspace_name":"kgovindmlwk","region":"eastus","compute_target":"govindk-nvidia-gpunode","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":30,"primary_metric":"accur

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
print(best_model)

Package:azureml-automl-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-core, training version:1.33.0, current version:1.32.0
Package:azureml-dataprep, training version:2.20.1, current version:2.18.0
Package:azureml-dataprep-native, training version:38.0.0, current version:36.0.0
Package:azureml-dataprep-rslex, training version:1.18.1, current version:1.16.1
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.32.0
Package:azureml-defaults, training version:1.33.0, current version:1.32.0
Package:azureml-interpret, training version:1.33.0, current version:1.32.0
Package:azureml-mlflow, training version:1.33.0, current version:1.32.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.32.0
Package:azureml-responsibleai, training version:1.33.0, current version:1.32.0
Package:azureml-telemetry, training version:1.33.0, current version:1.32.0
Package:azureml-train-automl-client, training version:1.33.0, current version:1.

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                ('StandardScalerWrapper',
                 StandardScalerWrapper(
    copy=True,
    with_mean=False,
    with_std=False
)),
                ('LogisticRegression',
                 LogisticRegression(C=51.79474679231202, class_weight=None,
                                    dual=False, fit_intercept=True,
                                    intercept_scaling=1, l1_ratio=None,
                                    max_iter=100, multi_class='ovr', n_jobs=1,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, 

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
model = best_automl_run.register_model(model_name = "best_automl.pkl", model_path = './outputs')
model.download(exist_ok=True) # downlaod the model artifacts with the conda environment YAML file

'outputs'

In [9]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=15)
environment = Environment.from_pip_requirements('govindk-automl-env', '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)
environment

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "govindk-automl-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-fo

In [10]:

service = model.deploy(
    workspace=ws,
    name="capstone-govindk-deploy",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-24 03:39:43+00:00 Creating Container Registry if not exists.
2021-08-24 03:39:44+00:00 Use the existing image.
2021-08-24 03:39:44+00:00 Generating deployment configuration.
2021-08-24 03:39:45+00:00 Submitting deployment to compute.
2021-08-24 03:39:49+00:00 Checking the status of deployment capstone-govindk-deploy..
2021-08-24 03:42:09+00:00 Checking the status of inference endpoint capstone-govindk-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [11]:
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
x = df.iloc[0, :-1].to_dict()
y = df.iloc[0, -1]

In [27]:
import json
import numpy as np
input_payload = json.dumps({
    'data': x
})
print(input_payload)
output = service.run(input_payload)
print(f"Predicted: {output}\nExpected: {y}")

{"data": {"age": 63.0, "sex": 1.0, "cp": 3.0, "trestbps": 145.0, "chol": 233.0, "fbs": 1.0, "restecg": 0.0, "thalach": 150.0, "exang": 0.0, "oldpeak": 2.3, "slope": 0.0, "ca": 0.0, "thal": 1.0}}
Predicted: [1]
Expected: 1


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
logs = service.get_logs()
print(logs)


2021-08-24T03:45:21,111539500+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-08-24T03:45:21,115672200+00:00 - rsyslog/run 
2021-08-24T03:45:21,118021900+00:00 - iot-server/run 
2021-08-24T03:45:21,128375800+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-24T03:45:21,407112800+00:00 - iot-server/finish 1 0
2021-08-24T03:45:21,412029700+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (62)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 94
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-24 03:45:25,527 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-24 03:45:25,529 | root | INFO | Starting up request id generator
2021-08-24 03:45:25,529 | root | INFO | Starting up app in

In [ ]:
service.delete()